In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
x=T.vector('x')

In [3]:
def square(x):
    return x*x

In [4]:
outputs,updates=theano.scan(fn=square,
                           sequences=x,
                           n_steps=x.shape[0])

In [5]:
square_op=theano.function(
inputs=[x],outputs=[outputs])

In [6]:
o_val=square_op(np.array([1,2,3,4,5]))

In [7]:
print (o_val)

[array([ 1.,  4.,  9., 16., 25.])]


In [24]:
x=np.array([1,2,3,4,5])

In [25]:
x.shape[0]

5

In [26]:
x[0].shape

()

In [27]:
x.shape

(5,)

In [28]:
#Recurrent network

In [58]:
def all_parity_pairs(nbit):
    N=2**nbit
    remainder=100-(N%100)
    Ntotal=N+remainder
    X=np.zeros((Ntotal,nbit))
    Y=np.zeros(Ntotal)
    for ii in range(Ntotal):
        i=ii%N
        for j in range(nbit):
            if i%(2**(j+1)) !=0:
                i -= 2**j
                X[ii,j]=1
        Y[ii]=X[ii].sum()%2
    return X,Y

In [15]:
x,y=all_parity_pairs(3)

In [20]:
x

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1

In [21]:
y

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.])

In [59]:
def weights(Mi,Mo):
    return np.random.randn(Mi,Mo)/np.sqrt(Mi+Mo)

In [60]:
class Hidden_layers:
    def __init__(self,M1,M2,an_id):
        self.id=an_id
        self.M1=M1
        self.M2=M2
        W=weights(self.M1,self.M2)
        b=np.zeros(self.M2)
        self.W=theano.shared(W,'W_%s' %self.id)
        self.b=theano.shared(b,'b_%s' %self.id)
        self.params=[self.W,self.b]
        
    def forward(self,X):
        return T.nnet.relu(X.dot(self.W)+self.b)

In [95]:
class ANN:
    def __init__(self,hidden_layer_sizes):
        self.hidden_layer_sizes=hidden_layer_sizes
        
    def forward(self,X):
        Z=X
        for h in self.hidden_layers:
            Z=h.forward(Z)
        return T.nnet.softmax(Z.dot(self.W)+self.b)
    
    def predict(self,X):
        return T.argmax(X,axis=1)
    
    def fit(self,X,Y,learning_rate=0.001,epochs=2,batch_size=5):
        Y=Y.astype(np.int32)
        N=X.shape[0]
        K=len(set(Y))
        self.hidden_layers=[]
        M1=N
        count=0
        for M2 in self.hidden_layer_sizes:
            h=Hidden_layers(M1,M2,count)
            self.hidden_layers.append(h)
            M1=M2
            count=count+1
            
        W=weights(M1,K)
        b=np.zeros(K)
        self.W=theano.shared(W,'W_logreg')
        self.b=theano.shared(b,'b_logreg')
        
        
        self.params=[self.W,self.b]
        for h in self.hidden_layers:
            self.params += h.params
            
        for h in self.params:
            print(h.shape)
        
        thX=T.matrix('X')
        thY=T.ivector('Y')
        pY=self.forward(thX)
        cost=-T.mean(T.log(pY[T.arange(thY.shape[0]),thY]))
        prediction=self.predict(pY)
        grads=T.grad(cost,self.params)
        
        updates=[
            (p,p-learning_rate*g) for p,g in zip(self.params,grads)
        ]
        
        train_op=theano.function(
        inputs=[thX,thY],
        outputs=[cost,prediction],
        updates=updates
        )
        
        n_batches=int(N/batch_size)
        costs=[]
        for i in range(epochs):
            X,Y=shuffle(X,Y)
            for j in range(n_batches):
                Xbatch=X[j*batch_size:(j*batch_size+batch_size)]
                Ybatch=Y[j*batch_size:(j*batch_size+batch_size)]
            
            c,p=train_op(Xbatch,Ybatch)
            
            costs.append(c)
            print ("i:",i,"cost:",c)
            
        plt.plot(costs)
        plt.show()
        
    
            
        
        
        
    

In [96]:

X,Y=all_parity_pairs(12)
model=ANN([20])
model.fit(X,Y)

Shape.0
Shape.0
Shape.0
Shape.0
820


ValueError: ('shapes (5,12) and (4100,20) not aligned: 12 (dim 1) != 4100 (dim 0)', (5, 12), (4100, 20))
Apply node that caused the error: Dot22(X, W_0)
Toposort index: 4
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]
Inputs shapes: [(5, 12), (4100, 20)]
Inputs strides: [(96, 8), (160, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Add}[(0, 0)](Dot22.0, InplaceDimShuffle{x,0}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/chetan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2662, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/home/chetan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2785, in _run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/chetan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2907, in run_ast_nodes
    if

In [77]:
X,Y=all_parity_pairs(12)
X.shape

(4100, 12)

In [78]:
Y.shape

(4100,)

In [80]:
N=X.shape[0]
N

4100

In [83]:
batch_size=5
n_batches=int(N/batch_size)
for i in range(20):
    X,Y=shuffle(X,Y)
    for j in range(n_batches):
            Xbatch=X[j*batch_size:(j*batch_size+batch_size)]
            Ybatch=Y[j*batch_size:(j*batch_size+batch_size)]
            print('Xbatch.shape',Xbatch.shape,'Ybatch.shape',Ybatch.shape)

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

...

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybatch.shape (5,)
Xbatch.shape (5, 12) Ybat

In [86]:
K=len(set(Y))
K

2